In [1]:
import os
import tensorflow_io as tfio
import pandas as pd
from sqlalchemy import create_engine
import datetime

In [2]:
%env TFIO_GOST_DATABASE_NAME=gost
%env TFIO_GOST_DATABASE_HOST=127.0.0.1:5050
%env TFIO_GOST_DATABASE_PORT=5432
%env TFIO_GOST_DATABASE_USER=postgres
%env TFIO_GOST_DATABASE_PASS=postgres

%env SQLA_GOST_DATABASE_NAME=gost
%env SQLA_GOST_DATABASE_HOST=localhost
%env SQLA_GOST_DATABASE_PORT=5432
%env SQLA_GOST_DATABASE_USER=postgres
%env SQLA_GOST_DATABASE_PASS=postgres

env: TFIO_GOST_DATABASE_NAME=gost
env: TFIO_GOST_DATABASE_HOST=127.0.0.1:5050
env: TFIO_GOST_DATABASE_PORT=5432
env: TFIO_GOST_DATABASE_USER=postgres
env: TFIO_GOST_DATABASE_PASS=postgres
env: SQLA_GOST_DATABASE_NAME=gost
env: SQLA_GOST_DATABASE_HOST=localhost
env: SQLA_GOST_DATABASE_PORT=5432
env: SQLA_GOST_DATABASE_USER=postgres
env: SQLA_GOST_DATABASE_PASS=postgres


In [ ]:
# TFIO connection
endpoint="postgresql://{}:{}@{}?port={}&dbname={}".format(
    os.environ['TFIO_GOST_DATABASE_USER'],
    os.environ['TFIO_GOST_DATABASE_PASS'],
    os.environ['TFIO_GOST_DATABASE_HOST'],
    os.environ['TFIO_GOST_DATABASE_PORT'],
    os.environ['TFIO_GOST_DATABASE_NAME'],
)

In [3]:
# SQLAlchemy + psycopg2 connection
sqla_endpoint ="postgresql+psycopg2://{}:{}@{}:{}/{}".format(
    os.environ['SQLA_GOST_DATABASE_USER'],
    os.environ['SQLA_GOST_DATABASE_PASS'],
    os.environ['SQLA_GOST_DATABASE_HOST'],
    os.environ['SQLA_GOST_DATABASE_PORT'],
    os.environ['SQLA_GOST_DATABASE_NAME'],
)
engine = create_engine(sqla_endpoint)

In [4]:
class Config():
    def __init__(self):
        table_names = {
            'strain1' : 'sensor_00158d00000e0ee5',
            'acc1' : 'sensor_00158d00000e0fe9',
            'acc2' : 'sensor_00158d00000e054c',
            'incl' : 'sensor_00158d00000e1024',
            'temp' : 'sensor_00158d00000e047b',
            'strain2': 'sensor_000000008bff436'
        }
        self.table_names = table_names
        
        self.column_names = {
            table_names['strain1'] : ['id','ts','ch_mv0','ch_mv1','ch_mv2','ch_mv3'],
            table_names['acc1'] : ['id','ts','ch_x','ch_y','ch_z'],
            table_names['acc2'] : ['id','ts','ch_x','ch_y','ch_z'],
            table_names['incl'] : ['id','ts','ch_x','ch_y'],
            table_names['temp'] : ['id','ts','ch_temperature'],
            table_names['strain2'] : ['id','ts','ch_mv0','ch_mv0_379']
        }

        self.schema = 'v1'        

In [5]:
class Settings():
    def __init__(self):
        self.sensors = ['acc1','acc2','incl']
        self.n_samples = 10
        self.start_date = datetime.datetime(2020,10,27)
        self.end_date = datetime.datetime(2020,10,28)

In [6]:
config = Config()
settings = Settings()

In [7]:
def generate_select():
    select_command = ''
    for sensor in settings.sensors:
        table = config.table_names[sensor]
        for column in config.column_names[table]:
            select_command+=config.schema+'.'+table+'.'+column+' ,' 
    return select_command[:-1]

def generate_where_id():
    where_clause = f"{config.schema}.{config.table_names[settings.sensors[0]]}.id < {settings.n_samples}"
    return where_clause

def generate_where():
    where_clause = f"{config.schema}.{config.table_names[settings.sensors[0]]}.ts BETWEEN \'{parse_date(settings.start_date)}\' AND \'{parse_date(settings.end_date)}\' "
    return where_clause

def generate_and():
    and_clause = ''
    if len(settings.sensors)>1:
        for i in range(len(settings.sensors)-1):
            and_clause += f"AND {config.schema}.{config.table_names[settings.sensors[0]]}.ts = {config.schema}.{config.table_names[settings.sensors[i+1]]}.ts "
            # ts ensures integrity in data
    return and_clause

def parse_date(date):
    
    return str(date)

def generate_query():
    query = ''
    query += f"SELECT {generate_select()}"
    query += f"FROM {config.schema}.{(', '+config.schema+'.').join([config.table_names[sensor] for sensor in settings.sensors])} "
    query += f"WHERE {generate_where()}"
    query += generate_and()
    return query



In [ ]:
query = 'SELECT strain.ch_mv0, strain.id FROM v1.sensor_000000008bff43b6 strain WHERE strain.id < 10'

In [ ]:
query =f"SELECT v1.sensor_00158d00000e0fe9.id AS qwe,v1.sensor_00158d00000e0fe9.ts ,v1.sensor_00158d00000e0fe9.ch_x ,v1.sensor_00158d00000e0fe9.ch_y ,v1.sensor_00158d00000e0fe9.ch_z ,v1.sensor_00158d00000e054c.id ,v1.sensor_00158d00000e054c.ts ,v1.sensor_00158d00000e054c.ch_x ,v1.sensor_00158d00000e054c.ch_y ,v1.sensor_00158d00000e054c.ch_z FROM v1.sensor_00158d00000e0fe9, v1.sensor_00158d00000e054c WHERE v1.sensor_00158d00000e0fe9.id < 10 AND v1.sensor_00158d00000e0fe9.id = v1.sensor_00158d00000e054c.id "

In [8]:
generate_query()

"SELECT v1.sensor_00158d00000e0fe9.id ,v1.sensor_00158d00000e0fe9.ts ,v1.sensor_00158d00000e0fe9.ch_x ,v1.sensor_00158d00000e0fe9.ch_y ,v1.sensor_00158d00000e0fe9.ch_z ,v1.sensor_00158d00000e054c.id ,v1.sensor_00158d00000e054c.ts ,v1.sensor_00158d00000e054c.ch_x ,v1.sensor_00158d00000e054c.ch_y ,v1.sensor_00158d00000e054c.ch_z ,v1.sensor_00158d00000e1024.id ,v1.sensor_00158d00000e1024.ts ,v1.sensor_00158d00000e1024.ch_x ,v1.sensor_00158d00000e1024.ch_y FROM v1.sensor_00158d00000e0fe9, v1.sensor_00158d00000e054c, v1.sensor_00158d00000e1024 WHERE v1.sensor_00158d00000e0fe9.ts BETWEEN '2020-10-27 00:00:00' AND '2020-10-28 00:00:00' AND v1.sensor_00158d00000e0fe9.ts = v1.sensor_00158d00000e054c.ts AND v1.sensor_00158d00000e0fe9.ts = v1.sensor_00158d00000e1024.ts "

In [ ]:
dataset = tfio.experimental.IODataset.from_sql(
    query = query,
    endpoint = endpoint
)

print(dataset.element_spec)

In [ ]:
df = pd.read_sql_table(
    table_name = 'sensor_00158d00000e0fe9',
    con = sqla_endpoint,
    schema = 'v1',
    index_col = 'id',
    parse_dates = 'ts',
    columns = ['ch_x','ch_y']
)

In [9]:
df = pd.read_sql_query(
    sql = generate_query(),
    con = sqla_endpoint,
    parse_dates = 'ts'
)

In [10]:
print(df)

          id                         ts     ch_x     ch_y     ch_z     id  \
0        133 2020-10-27 07:59:58.000000  0.08004  0.37594  0.92080  17866   
1        134 2020-10-27 07:59:58.030303  0.07784  0.37548  0.92148  17867   
2        135 2020-10-27 07:59:58.060606  0.07776  0.37532  0.92095  17868   
3        136 2020-10-27 07:59:58.090909  0.07996  0.37594  0.91966  17869   
4        137 2020-10-27 07:59:58.121212  0.07768  0.37601  0.92239  17870   
...      ...                        ...      ...      ...      ...    ...   
73538  78570 2020-10-27 23:59:59.878788  0.07867  0.37563  0.92209  96438   
73539  78571 2020-10-27 23:59:59.909091  0.07761  0.37510  0.92178  96439   
73540  78572 2020-10-27 23:59:59.939394  0.07966  0.37701  0.92459  96440   
73541  78573 2020-10-27 23:59:59.969697  0.07898  0.37617  0.92285  96441   
73542  78574 2020-10-28 00:00:00.000000  0.07784  0.37563  0.92216  96442   

                              ts     ch_x     ch_y     ch_z     id  \
0    